In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local").appName("ScenarioBasedQtns").getOrCreate()

24/05/29 18:24:26 WARN Utils: Your hostname, codespaces-f443ca resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/05/29 18:24:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/29 18:24:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
sc = spark.sparkContext

In [8]:
rdd_in = sc.textFile("/workspaces/myspark/data/titanic.csv")


In [11]:
rdd_in.take(4)

['PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked',
 '1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S',
 '2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C',
 '3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S']

In [ ]:
rdd_fltr=rdd_in.filter(lambda x: not x."PassengerId")